In [5]:
!pip install pymdptoolbox

import numpy as np
import mdptoolbox

# -------------------------------
# State and Action definitions
# -------------------------------

# X, Y, Theta discretization
X_vals = np.arange(-16, 17)  # 33 values
Y_vals = np.arange(-10, 23)  # 33 values
theta_vals = np.array([0, 45, 90, 135, 180, 225, 270, 315])  # 8 values
leg_vals = [0, 1]  # left/right leg contact

# Actions
ACTIONS = ['noop', 'left', 'main', 'right']
nA = len(ACTIONS)

# -------------------------------
# Ground surface and forbidden states
# -------------------------------

def Y_ground(X):
    if -16 <= X <= -9:
        return -4
    elif X == -8:
        return -3
    elif X == -7:
        return -2
    elif X == -6:
        return -1
    elif -5 <= X <= 7 or X in [15, 16]:
        return 0
    elif 8 <= X <= 9 or X == 14:
        return 1
    elif 10 <= X <= 11 or X == 13:
        return 2
    elif X == 12:
        return 3
    else:
        return -10  # outside viewport

def is_forbidden(x, y):
    return y < Y_ground(x)

# -------------------------------
# Encode state as index
# -------------------------------
state_map = {}
state_list = []
idx = 0
for x in X_vals:
    for y in Y_vals:
        for th in theta_vals:
            for l in leg_vals:
                for r in leg_vals:
                    if not is_forbidden(x, y):
                        state_map[(x, y, th, l, r)] = idx
                        state_list.append((x, y, th, l, r))
                        idx += 1
nS = len(state_list)
print("Number of valid states:", nS)

# -------------------------------
# COM update tables for main engine
# -------------------------------
main_update = {
    0:  (2, -1), 45:  (1, 0), 315: (1, -2), 90:  (0, 1),
    270: (0, -3), 135: (-1, 0), 225: (-1, -2), 180: (-2, -1)
}

# Left engine updates (theta changes -45)
left_update = {
    0:  (0, -3, -45), 45: (1, -2, -45), 315:(-1,-2,-45), 90:(2,-1,-45),
    270:(-2,-1,-45), 135:(1,0,-45), 225:(-1,0,-45), 180:(0,1,-45)
}

# Right engine updates (theta changes +45)
right_update = {
    0:  (0, 1, 45), 45: (-1,0,45), 315:(1,0,45), 90:(-2,-1,45),
    270:(2,-1,45), 135:(-1,-2,45), 225:(1,-2,45), 180:(0,-3,45)
}

# -------------------------------
# Transition probability matrix and rewards
# -------------------------------
P = [np.zeros((nS, nS)) for _ in range(nA)]
R = [np.zeros((nS, nS)) for _ in range(nA)]

def normalize_theta(th):
    th = th % 360
    if th not in theta_vals:
        th = min(theta_vals, key=lambda x: abs(x - th))
    return th

def step_reward(x, y, th, l, r, action):
    R_pos = - abs(x)
    R_orient = - (abs(th - 90)//45)
    R_leg = 10*(l+r)
    if action == 'main':
        R_eng = -0.3
    elif action in ['left', 'right']:
        R_eng = -0.03
    else:
        R_eng = 0
    return R_pos + R_orient + R_leg + R_eng

# -------------------------------
# Construct transition and reward matrices
# -------------------------------
for s_idx, (x, y, th, l, r) in enumerate(state_list):
    for a_idx, a in enumerate(ACTIONS):
        # Default motion: gravity (noop)
        x_new, y_new, th_new = x, y-1, th
        if a == 'main':
            dx, dy = main_update[th]
            x_new = x + dx
            y_new = y + dy
        elif a == 'left':
            dx, dy, dtheta = left_update[th]
            x_new = x + dx
            y_new = y + dy
            th_new = normalize_theta(th + dtheta)
        elif a == 'right':
            dx, dy, dtheta = right_update[th]
            x_new = x + dx
            y_new = y + dy
            th_new = normalize_theta(th + dtheta)

        # Apply gravity after thrust
        if a != 'noop':
            y_new -= 1

        # Clamp to bounds
        x_new = min(max(x_new, X_vals[0]), X_vals[-1])
        y_new = min(max(y_new, Y_vals[0]), Y_vals[-1])

        # Leg contact logic
        l_new = 1 if y_new <= Y_ground(x_new) else 0
        r_new = l_new

        # Forbidden check
        if is_forbidden(x_new, y_new):
            s_new_idx = s_idx
        else:
            s_new_idx = state_map.get((x_new, y_new, th_new, l_new, r_new), s_idx)

        # Deterministic transition
        P[a_idx][s_idx, s_new_idx] = 1.0
        R[a_idx][s_idx, s_new_idx] = step_reward(x_new, y_new, th_new, l_new, r_new, a)

# -------------------------------
# Solve MDP using Value Iteration
# -------------------------------
vi = mdptoolbox.mdp.ValueIteration(P, R, 0.95)
vi.run()


print("\n" + "="*60)
print("LUNAR LANDER MDP — VALUE ITERATION SUMMARY")
print("="*60)

print(f"Valid states      : {nS}")
print(f"Actions available : {nA} → {ACTIONS}")
print(f"Policy length     : {len(vi.policy)}\n")

# Print policy as comma-separated action indices
policy_indices = ",".join(map(str, vi.policy))
print("Policy (action indices):")
print(policy_indices + "\n")

# Print policy as comma-separated action names
policy_actions = ",".join([ACTIONS[a] for a in vi.policy])
print("Policy (interpreted actions):")
print(policy_actions + "\n")

# Print value function as comma-separated values
values_str = ",".join([f"{v:.3f}" for v in vi.V])
print("Value function (V):")
print("V = " + values_str + "\n")

print("="*60)
print("Action index mapping:")
for i, a in enumerate(ACTIONS):
    print(f"  {i} → {a}")
print("="*60 + "\n")



Number of valid states: 25120

LUNAR LANDER MDP — VALUE ITERATION SUMMARY
Valid states      : 25120
Actions available : 4 → ['noop', 'left', 'main', 'right']
Policy length     : 25120

Policy (action indices):
3,3,3,3,3,3,3,3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3,3,3,3,0,0,0,0,3,3,3,3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3,3,3,3,2,2,2,2,2,2,2,2,1,1,1,1,1,1,1,1,2,2,2,2,1,1,1,1,3,3,3,3,3,3,3,3,2,2,2,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3,3,3,3,3,3,3,3,2,2,2,2,2,2,2,2,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,3,3,3,3,3,3,3,3,2,2,2,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3,3,3,3,3,3,3,3,3,3,3,3,2,2,2,2,2,2,2,2,1,1,1,1,1,1,1,1,1,1,1,1,3,3,3,3,3,3,3,3,3,3,3,3,2,2,2,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3,3,3,3,3,3,3,3,3,3,3,3,2,2,2,2,2,2,2,2,1,1,1,1,1,1,1,1,1,1,1,1,3,3,3,3,3,3,3,3,3,3,3,3,2,2,2,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3,3,3,3,3,3,3,3,3,3,3,3,2,2,2,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3,3,3,3,3,3,3,3,3,3,3,3,2,2,2,2,2,2,2,2,1,1,1,1,1,1,1,1,1,1,1,1,3,3,3,3,3,3,3,3,3,3,3,3,2,2,2,2,1,1,1,1,1,1,1,